# 1. 데이터 준비

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')
wine.head()
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)
print(train_data.shape, test_data.shape)

(5197, 3) (1300, 3)


# 2. Random Forest 모델 학습
- 결정 트리의 앙상블. DecisionTreeClassifier()가 제공하는 중요한 매개변수를 모두 제공.
- 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고리즘

- **교차 검증**시 `return_train_scres = True` 매개변수 설정: 훈련 세트에 대한 점수도 반환해주는 매개변수 설정 값
- 훈련 세트에 대한 점수와 검증 세트에 대한 점수를 함께 보면 과적합 여부를 쉽게 알 수 있다.

## 2-1. 첫 번째 랜덤 포레스트 모델 학습

In [2]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1, random_state = 42)
rf.fit(train_data, train_target)

RandomForestClassifier(n_jobs=-1, random_state=42)

**교차 검증 `cross_validate()` 함수 이용**

In [3]:
from sklearn.model_selection import cross_validate
scores = cross_validate(rf, train_data, train_target,
                       return_train_score = True, n_jobs = -1)

**검증 세트의 정확도 확인**

In [4]:
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # overfitting

0.9973541965122431 0.8905151032797809


**특징 중요도 `feature_importances`**

In [5]:
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


- 결정 트리를 사용했을 때의 결과와 비교해보면 두 번째 특성인 당도의 중요도가 감소하고 알코올 도수와 pH 특성의 중요도가 조금 상승했다 이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다. 따라서 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. 결과적으로 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 된다.

## 2-2. 두 번째 랜덤 포레스트 모델 학습
- 부트스트랩 샘플을 이용함 (샘플 중복 추출)
    - `oob_score = True`: 부트스트랩 샘플에 포함되지 않고 남은 샘플(OOB)을 사용해 결정 트리를 평가할 수 있다. 교차 검증을 대신할 수 있어서 더 많은 샘플을 사용할 수 있게 되는 셈이다.

In [6]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)
rf.fit(train_data, train_target)

RandomForestClassifier(n_jobs=-1, oob_score=True, random_state=42)

**OOB 샘플을 이용한 교차 검증 정확도 점수 확인**

In [7]:
print(rf.oob_score_) # 교차 검증에서 얻은 점수와 매우 비슷 

0.8934000384837406


## 2-3. 테스트 세트의 정확도 확인

In [8]:
rf.score(test_data, test_target)

0.8892307692307693

# 3. Extra Tree 모델학습
- 기본적으로 100개의 트리를 훈련함
- 일부 특성을 랜덤하게 선택해 노드를 분할하고, 무작위로 분할하기 때문에 계산 속도가 빠르다.
- 랜덤 포레스트와 비슷하지만 부트스트랩 샘플을 사용하지 않는다는 차이점
- 엑스트라 트리가 무작위성이 크기 때문에 랜덤 포레스트보다 더 많은 결정트리를 훈련해야 한다.
- `splitter = 'random'`: 엑스트라 트리가 사용하는 결정 트리 
- 하나의 트리에서 무작위 특성 분할을 하면 성능이 낮아지지만 많은 트리를 앙상블하기 때문에 과대적합을 막고 검증 점수를 높이는 효과가 있다.

In [9]:
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)
et.fit(train_data, train_target)

ExtraTreesClassifier(n_jobs=-1, random_state=42)

**교차 검증 `cross_validate()` 함수 이용**

In [10]:
from sklearn.model_selection import cross_validate
scores = cross_validate(et, train_data, train_target
                       , return_train_score = True, n_jobs = -1)

**검증 세트의 정확도 확인**

In [11]:
print(np.mean(scores['train_score']), np.mean(scores['test_score'])) # overfitting

0.9974503966084433 0.8887848893166506


**특징 중요도 `feature_importances`**

In [12]:
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 3-1. 테스트 세트의 정확도 확인

In [13]:
et.score(test_data, test_target)

0.8861538461538462